In [11]:

import requests
from config import geoapify_key 
import csv
import pandas as pd
import pprint
import json
import time 



In [12]:
# get the data from the json file
json_path = "./Resources/IMDB_API_100_location.json"
# json_path = "./Resources/IMDB_API_100_location.json"
df = pd.read_json(json_path)
# remove the duplicate rows by location_name
df = df.drop_duplicates(subset=['location_name'], keep='first')

# create a new list to store the data
location_list = []
df.head(10)



,id,location_id,location_name
0,tt11858890,/title/tt11858890/filminglocations/lc2749017,"Bangkok, Thailand"
1,tt11858890,/title/tt11858890/filminglocations/lc2763749,"Pinewood Studios, Pinewood Road, Iver Heath, I..."
2,tt11858890,/title/tt11858890/filminglocations/lc2749018,Thailand
3,tt11858890,/title/tt11858890/filminglocations/lc3578189,"Siem Reap, Cambodia"
4,tt15009428,/title/tt15009428/filminglocations/lc3375865,"London, England, UK"
5,tt15009428,/title/tt15009428/filminglocations/lc3375866,Greece
6,tt17024450,/title/tt17024450/filminglocations/lc2973962,"Minori, Campania, Italy"
7,tt17024450,/title/tt17024450/filminglocations/lc2973960,"Atrani, Campania, Italy"
8,tt17024450,/title/tt17024450/filminglocations/lc3493652,"Maiori, Campania, Italy"
9,tt17024450,/title/tt17024450/filminglocations/lc2973961,"Ravello, Campania, Italy"


In [13]:
# loop through the rows and get the location data


def findthelocation(name):
    locationinfo = {}
    locationinfo["location_name"] = name
    name = name.replace("UK","United Kingdom")
    name = name.replace("USA","United States of America")

    
    url = "https://api.geoapify.com/v1/geocode/autocomplete?text="+name+"&format=json&apiKey="+geoapify_key
    
    response = requests.get(url)
    
    
    if len(response.json()['results'])==0:
    
        if "," in name:
            name = name.split(",",1)[1]
            return findthelocation(name)
             
        else:
            locationinfo["lat"] = "N/A"
            locationinfo["lon"] = "N/A"
          
            return locationinfo
        
    else:
        locationinfo["lat"] = response.json()["results"][0]["lat"]
        locationinfo["lon"] = response.json()["results"][0]["lon"]

        
    return locationinfo





In [14]:
def check_value_in_list_of_dicts(list_of_dicts, value):
    
    for dictionary in list_of_dicts:

        if value in dictionary.get('location_name'):
            return True
    return False

In [15]:
jsonpath='./Resources/IMDB_API_100_location_geoapify.json'
with open(jsonpath, 'r') as file:
    data = json.load(file)

filtered_data = [item for item in data if 'lon' in item]

with open(jsonpath, 'w') as file:
    json.dump(filtered_data, file, indent=4)

In [16]:
i=0
for index, row in df.iterrows():
    i=i+1
    print(i)
    
# for index, row in df.iterrows():
    # time.sleep(0.5)
    text = row["location_name"]

    with open(jsonpath, 'r') as file:
        data = json.load(file)
        if not isinstance(data, list):
            data = [data]
            data = [item for item in data if 'lon' in item]
        if check_value_in_list_of_dicts(data,text):
            print('We have it')   
        else:
            time.sleep(0.5)
            location=findthelocation(text)    
            print(text)
            pprint.pprint(location)
            with open(jsonpath, 'w') as file:
                
                data.append(location)
                json.dump(data, file, indent=4)
                print("add a row")
            
    # location_list.append(location)





1
We have it
2
We have it
3
We have it
4
We have it
5
We have it
6
We have it
7
Minori, Campania, Italy
{'lat': 40.652708, 'location_name': 'Minori, Campania, Italy', 'lon': 14.624941}
add a row
8
Atrani, Campania, Italy
{'lat': 40.636908, 'location_name': 'Atrani, Campania, Italy', 'lon': 14.607541}
add a row
9
Maiori, Campania, Italy
{'lat': 40.65235025,
 'location_name': 'Maiori, Campania, Italy',
 'lon': 14.642910775796539}
add a row
10
Ravello, Campania, Italy
{'lat': 40.6496643,
 'location_name': 'Ravello, Campania, Italy',
 'lon': 14.6122753}
add a row
11
We have it
12
Napoli, Campania, Italy
{'lat': 40.8358846,
 'location_name': 'Napoli, Campania, Italy',
 'lon': 14.2487679}
add a row
13
We have it
14
We have it
15
We have it
16
We have it
17
We have it
18
We have it
19
We have it
20
We have it
21
We have it
22
We have it
23
We have it
24
We have it
25
We have it
26
We have it
27
We have it
28
We have it
29
We have it
30
We have it
31
We have it
32
We have it
33
We have it
34
W

KeyboardInterrupt: 

In [ ]:
# # create a new dataframe
# location_df = pd.DataFrame(location_list)
# location_df.head()
# # reorder the columns
# location_df = location_df[["location_name","lat","lon"]]
# # save the dataframe to csv file
# location_df.to_csv("./Resources/IMDB_API_100_location_geoapify_test.csv", index=False)
# location_df.to_json("./Resources/IMDB_API_100_location_geoapify_test.json", orient="records")
